In [1]:
import os,random
import csv
os.environ["KERAS_BACKEND"] = "tensorflow"
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
from copy import deepcopy
from tensorflow.keras.models import load_model
import tensorflow.keras.models as models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, AlphaDropout, GlobalAveragePooling1D, BatchNormalization, add
from tensorflow.python.keras.layers.core import Flatten, Dense, Dropout, Reshape, Activation,Lambda,Permute
from tensorflow.python.keras.layers.convolutional import Conv1D,Conv2D, AveragePooling1D, ZeroPadding2D, Convolution2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.regularizers import *
from tensorflow.keras import layers
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import pickle,time
import random, sys
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
mirrored_strategy = tf.distribute.MirroredStrategy()
from art.attacks.evasion import CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import KerasClassifier
tf.compat.v1.disable_eager_execution()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# Dataset setup
Xd = pickle.load(open('RML22.pickle', 'rb'))

snrs,mods = map(lambda j: sorted( list( set( map( lambda x: x[j], Xd.keys() ) ) ) ), [1,0])

X = []
Y_snr = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
    Y_snr = Y_snr + [mod]*120000
X = np.vstack(X)
clip_value = max(abs(float(np.min(X))), float(np.max(X)))
print(X.shape)
Y_snr = np.vstack(Y_snr)

(1260000, 2, 128)


In [4]:
def to_onehot(yin):
    yy = list(yin)
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_snr = to_onehot(map(lambda x: mods.index(lbl[x][0]), range(X.shape[0])))

In [5]:
# Partition the dataset into training and testing datasets
np.random.seed(2022)
n_examples = X.shape[0]
n_train    = int(round(n_examples * 0.5))
train_idx  = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx   = list(set(range(0,n_examples))-set(train_idx))
X_train    = X[train_idx]
X_test     = X[test_idx]

Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

input_shp = list(X_train.shape[1:])
output_shp = Y_train.shape[1]
print(f'Network input shape in variable input-shp: {input_shp}')
print(f'Network output shape in variable input-shp: {output_shp}')
print(X_train.shape, input_shp)
classes = mods

Network input shape in variable input-shp: [2, 128]
Network output shape in variable input-shp: 10
(630000, 2, 128) [2, 128]


In [6]:
X_train_orig = deepcopy(X_train)
X_test_orig = deepcopy(X_test)

num_samples_list = [16, 32, 64]
field_names= ['num_subsample'] + list(snrs)
acc_list = []

In [ ]:
subSampling_schemes = ["complex_cnn", "resnet", "cldnn", "holistic", "uniform"]

for subSampling_scheme in subSampling_schemes:

    for num_samples in num_samples_list:

        # Data-driven subsampling
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        X_train = deepcopy(X_train_orig)
        X_test =  deepcopy(X_test_orig)


        if subSampling_scheme == "holistic":
            cldnn = load_model('ranker_cldnn.wts.h5')
            cnn = load_model('ranker_complex_cnn.wts.h5')
            resnet = load_model('ranker_resnet.wts.h5')  
        elif subSampling_scheme != "uniform":
            orig_model = load_model('ranker_' + subSampling_scheme + '.wts.h5')

        X_train_sub = np.zeros((X_train.shape[0], X_train.shape[1], num_samples))
        X_test_sub = np.zeros((X_test.shape[0], X_test.shape[1], num_samples))
        print(X_train_sub.shape)

        for i, snr in enumerate(snrs):
            print('SNR = ', snr)
            #train data
            train_SNRs = list(map(lambda x: lbl[x][1], train_idx))
            orig_idxs = np.where(np.array(train_SNRs) == snr)[0]
            snr_data = X_train[orig_idxs]
            snr_out = Y_train[orig_idxs]
            snr_acc_list = []
            snr_data_copy = deepcopy(snr_data)
            # print(f"Number of train data with SNR = {snr} is {orig_idxs.shape}")

            #test Data
            test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
            orig_idxs_test = np.where(np.array(test_SNRs)== snr)[0]
            test_data =  X_test[orig_idxs_test]
            # print(f"Number of test data with SNR = {snr} is {orig_idxs_test.shape}")

            if subSampling_scheme == "holistic":
                #~~~~~~~~~~~~~~~~Holistic~~~~~~~~~~~~~~~~~~~~~~~~~~~
                holistic_list = []
                try:
                    with open('./data/resnet_index/indexList_' + str(snr)+ '.data', 'rb') as fp:
                        resnet_list = pickle.load(fp)
                    with open('./data/complex_cnn_index/indexList_' + str(snr)+ '.data', 'rb') as fp:
                        cnn_list = pickle.load(fp)
                    with open('./data/cldnn_index/indexList_' + str(snr)+ '.data', 'rb') as fp:
                        cldnn_list = pickle.load(fp)

                except:
                    cldnn_list = []
                    cnn_list = []
                    resnet_list = []
                    snr_data_copy = deepcopy(snr_data)
                    for idx in range(X.shape[2]):
                        snr_data = deepcopy(snr_data_copy)
                        snr_data = snr_data.transpose((2, 1, 0))
                        new_snr_data = np.append(snr_data[:idx], np.zeros((1, snr_data.shape[1], snr_data.shape[2])), axis=0)
                        snr_data = np.append(new_snr_data, snr_data[idx+1:], axis=0)
                        snr_data = snr_data.transpose((2, 1, 0))

                        cldnn_score = cldnn.evaluate(snr_data, snr_out, batch_size=2048, verbose=0)
                        cldnn_list.append((idx, cldnn_score[1]))
                        cnn_score = cnn.evaluate(snr_data, snr_out, batch_size=2048, verbose=0)
                        cnn_list.append((idx, cnn_score[1]))
                        resnet_score = resnet.evaluate(snr_data, snr_out, batch_size=2048, verbose=0)
                        resnet_list.append((idx, resnet_score[1]))

                    with open('./data/resnet_index/indexList_' + str(snr)+ '.data', 'wb') as fp:
                        pickle.dump(resnet_list, fp)
                    with open('./data/complex_cnn_index/indexList_' + str(snr)+ '.data', 'wb') as fp:
                        pickle.dump(cnn_list, fp)
                    with open('./data/cldnn_index/indexList_' + str(snr)+ '.data', 'wb') as fp:
                        pickle.dump(cldnn_list, fp)

                # tier 3
                cldnn_list.sort(key=lambda x: x[1])
                cldnn_list = cldnn_list[:num_samples]
                cldnn_dict = {k: v for k, v in cldnn_list}

                cnn_list.sort(key=lambda x: x[1])
                cnn_list = cnn_list[:num_samples]
                cnn_dict = {k: v for k, v in cnn_list}

                resnet_list.sort(key=lambda x: x[1])
                resnet_list = resnet_list[:num_samples]
                resnet_dict = {k: v for k, v in resnet_list}

                # tier 2
                cldnn_cnn_list = [(k, cldnn_dict[k] + cnn_dict[k]) for k in set(cldnn_dict).intersection(cnn_dict)]
                cldnn_cnn_dict = {k: v for k, v in cldnn_cnn_list}
                cnn_resnet_list = [(k, cnn_dict[k] + resnet_dict[k]) for k in set(cnn_dict).intersection(resnet_dict)]
                cnn_resnet_dict = {k: v for k, v in cnn_resnet_list}
                cldnn_resnet_list = [(k, cldnn_dict[k] + resnet_dict[k]) for k in set(cldnn_dict).intersection(resnet_dict)]
                cldnn_resnet_dict = {k: v for k, v in cldnn_resnet_list}

                # tier 1
                cldnn_cnn_resnet_list = [(k, cldnn_cnn_dict[k] + resnet_dict[k]) for k in set(cldnn_cnn_dict).intersection(resnet_dict)]
                cldnn_cnn_resnet_list.sort(key=lambda x: x[1])

                tier_1_samples = cldnn_cnn_resnet_list
                tier_1 = [ele[0] for ele in tier_1_samples]


                # remove tier 1 entries from tier 2
                for ele in tier_1:
                    del cldnn_cnn_dict[ele]
                    del cnn_resnet_dict[ele]
                    del cldnn_resnet_dict[ele]

                cldnn_cnn_list = [(k, v) for k, v in cldnn_cnn_dict.items()]
                cnn_resnet_list = [(k, v) for k, v in cnn_resnet_dict.items()]
                cldnn_resnet_list = [(k, v) for k, v in cldnn_resnet_dict.items()]

                tier_2_samples = cldnn_cnn_list + cnn_resnet_list + cldnn_resnet_list
                tier_2_samples.sort(key=lambda x: x[1])

                tier_2 = [ele[0] for ele in tier_2_samples]


                # remove tier 1 & tier 2 entries from tier 3
                for ele in tier_1:
                    del cldnn_dict[ele]
                    del cnn_dict[ele]
                    del resnet_dict[ele]

                for ele in tier_2:
                    try: del cldnn_dict[ele]
                    except: pass
                    try: del cnn_dict[ele]
                    except: pass
                    try: del resnet_dict[ele]
                    except: pass

                cldnn_list = [(k, v) for k, v in cldnn_dict.items()]
                cnn_list = [(k, v) for k, v in cnn_dict.items()]
                resnet_list = [(k, v) for k, v in resnet_dict.items()]

                tier_3_samples = cldnn_list + cnn_list + resnet_list
                tier_3_samples.sort(key=lambda x: x[1])

                holistic_list = holistic_list + tier_1_samples
                holistic_list = holistic_list + tier_2_samples
                holistic_list = holistic_list + tier_3_samples
                holistic_list = holistic_list[:num_samples]
                holistic_list.sort(key=lambda x: x[1])
                holistic_list.sort(key=lambda x: x[0])

                snr_idxs = [ele[0] for ele in holistic_list]
                #~~~~~~~~~~~~~~~~Holistic~~~~~~~~~~~~~~~~~~~~~~~~~~~

            elif subSampling_scheme == "uniform":
                #~~~~~~~~~~~~~~~~Uniform~~~~~~~~~~~~~~~~~~~~~~~~~~~
                snr_idxs = list(range(1, 128, int(128/num_samples)))
                #~~~~~~~~~~~~~~~~Uniform~~~~~~~~~~~~~~~~~~~~~~~~~~~

            else: 
                #~~~~~~~~~~~~~~~~ResNet/CNN/CLDNN~~~~~~~~~~~~~~~~~~~~~~~~~~~
                try:
                    with open('./data/' + subSampling_scheme + '_index/indexList_' + str(snr)+ '.data', 'rb') as fp:
                        snr_acc_list = pickle.load(fp)
                except:
                    # only using train data for subsampling
                    for idx in range(X_train.shape[2]):
                        snr_data = deepcopy(snr_data_copy)
                        snr_data = snr_data.transpose((2,1,0))
                        new_snr_data = np.append(snr_data[:idx], np.full((1, snr_data.shape[1], snr_data.shape[2]), np.mean(snr_data[idx])), axis=0)
                        snr_data = np.append(new_snr_data, snr_data[idx+1:], axis=0)
                        snr_data = snr_data.transpose((2,1,0))
                        score = orig_model.evaluate(snr_data, snr_out, batch_size=len(orig_idxs), verbose=0)
                        snr_acc_list.append((idx, score))

                    with open('./data/' + subSampling_scheme + '_index/indexList_' + str(snr)+ '.data', 'wb') as fp:
                        pickle.dump(snr_acc_list, fp)

                snr_acc_list.sort(key=lambda x: x[1])
                snr_acc_list = snr_acc_list[:num_samples]
                snr_acc_list.sort(key=lambda x: x[0]) 
                snr_idxs = [ele[0] for ele in snr_acc_list]
                #~~~~~~~~~~~~~~~~ResNet/CNN/CLDNN~~~~~~~~~~~~~~~~~~~~~~~~~~~

            #train data
            snr_data = snr_data.transpose((2,1,0))
            snr_data = snr_data[snr_idxs]
            snr_data = snr_data.transpose((2,1,0))

            #test data
            test_data = test_data.transpose((2,1,0))
            test_data = test_data[snr_idxs]
            test_data = test_data.transpose((2,1,0))

            X_train_sub[orig_idxs] = snr_data
            X_test_sub[orig_idxs_test] = test_data

        X_train = X_train_sub
        X_test = X_test_sub
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        print(X_train.shape)
        print(Y_train.shape)

        in_shp = list(X_train.shape[1:])
        print(X_train.shape, in_shp, snrs)
        classes = mods

        # build model ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        def residual_stack(x):
            def residual_unit1(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)

                y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit2(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                          kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit3(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit4(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            x = layers.Conv1D(8, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                      kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)

            x = residual_unit1(x)
            x = layers.Conv1D(16, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit2(x)
            x = layers.Conv1D(32, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit3(x)
            x = layers.Conv1D(64, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit4(x)
            x = layers.AveragePooling1D(data_format='channels_first',pool_size=in_shp[1])(x)
            return x
    
        inputs=layers.Input(shape=in_shp)
        x = residual_stack(inputs)
        x = Flatten()(x)
        x = Dense(256,kernel_initializer="he_normal", activation="selu", name="dense1")(x)
        x = AlphaDropout(0.5)(x)
        x = Dense(len(classes),kernel_initializer="he_normal", activation="softmax", name="dense3")(x)
        model = models.Model(inputs,x)
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
        model.summary()

        nb_epoch = 200     # number of epochs to train on
        batch_size = 512  # training batch size

        # Train the Model
        filepath = './data/model/subsample_' + subSampling_scheme + '_' + str(num_samples) + '.wts.h5'
        model.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adam(lr=0.001),
                    metrics=['accuracy'])

        history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=2, validation_split=0.2, 
                callbacks = [
                    keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='auto')
                    ,keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, verbose=0, mode='auto')
                ])
        model.load_weights(filepath)

        # Evaluate and Plot Model Performance
        score = model.evaluate(X_test, Y_test, verbose=0, batch_size=batch_size)
        print(score)
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # Evaluate classification accuracy per SNR and store it.
        field_names= ['num_subsample'] + list(snrs)
        acc = {}
        acc["num_subsample"] = num_samples
        for snr in snrs:
            # extract classes @ SNR
            print(snr)
            test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
            test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
            test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]

            test_Y_i_hat = model.predict(test_X_i)
            conf = np.zeros([len(classes),len(classes)])
            confnorm = np.zeros([len(classes),len(classes)])
            for i in range(0,test_X_i.shape[0]):
                j = list(test_Y_i[i,:]).index(1)
                k = int(np.argmax(test_Y_i_hat[i,:]))
                conf[j,k] = conf[j,k] + 1
            for i in range(0,len(classes)):
                confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])  
            cor = np.sum(np.diag(conf))
            ncor = np.sum(conf) - cor
            print ("Overall Accuracy: ", cor / (cor+ncor))
            acc[snr] = 1.0*cor/(cor+ncor)

        print (acc)
        with open('./results/acc_subsampling_' + subSampling_scheme + '' + str(num_samples) + '.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc)
        acc_list.append(acc)

    with open('./results/acc_subsampling_' + subSampling_scheme + '_all.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()
        for acc_i in acc_list:
            writer.writerow(acc_i)